# Layout Plotting

In [ ]:
import os
import numpy as np

from impact import Impact

from impact.parsers import parse_impact_input
from impact.plotting import layout_plot
from impact.fieldmaps import ele_field, lattice_field

from bokeh.plotting import output_notebook
from bokeh.plotting import show


import matplotlib.pyplot as plt
from ipywidgets import interact

output_notebook(hide_banner=True)

In [ ]:
infile = "templates/lcls_injector/ImpactT.in"

## matplotlib plotting from the Impact object

In [ ]:
I = Impact(infile)
I.plot(figsize=(12, 4))

In [ ]:
# Track 1 particle
I.track1()

# These are the fields seen by the bunch centroid
I.plot("Ez", y2="Bz")

In [ ]:
# Fieldmaps can be added to existing axes

from impact.plot import add_fieldmaps_to_axes

fig, ax = plt.subplots()
add_fieldmaps_to_axes(I, t=1, ax=ax, bounds=(0, 10))

# Low-level get field

In [ ]:
# Get fieldmaps
fmaps = I.fieldmaps

# This shows the field from a single element
ele_field(I.ele["GUN"], z=0.1, fmaps=fmaps)

In [ ]:
# this gets the total field from all elements in the lattice
lattice_field(I.lattice, z=0.3, fmaps=fmaps)

In [ ]:
I.input["fieldmaps"].keys()

In [ ]:
# Period in ps
T = 1 / 2856000000.0
T / 1e-12  # ps

In [ ]:
I = Impact(infile)
I.plot(figsize=(12, 4), field_t=0)

In [ ]:
# Make a function to get a field array at a given phase/2pi

zz = np.linspace(0, I.stop, 1000)


def field_at(phi0):
    Ez_at = np.array([I.field(z=z, t=phi0 / 2856000000.0, component="Ez") for z in zz])
    plt.plot(zz, Ez_at / 1e6)
    plt.ylim(-100, 100)
    # plt.xlim(2, 2.2)


# def field_at(phi0):
#    t=phi0/2856000000.0
#    fig = I.plot(figsize=(12,4), field_t=t, return_figure=True)
#    fig.axes[0].set_ylim(-100,100)
#    fig.axes[1].set_ylim(-0, 0.3)

field_at(0)

In [ ]:
# Visualize the oscillating fields
interact(field_at, phi0=(0, 2, 0.01))

In [ ]:
# These are the fields seen by the bunch
I.track1()
zz = I.stat("mean_z")
tt = I.stat("t")

Ez_seen = np.array(
    [
        lattice_field(I.lattice, z=z, t=t, fmaps=fmaps, component="Ez")
        for z, t in zip(zz, tt)
    ]
)
zz = zz[: len(tt)]
plt.plot(zz, Ez_seen)

In [ ]:
# MeV energy gain
-np.trapz(Ez_seen, zz) / 1e6

In [ ]:
# This is pretty close to the tracked kinetic energy
I.stat("mean_kinetic_energy")[-1] / 1e6

## Bokeh plotting from the input file

In [ ]:
infile = "templates/lcls_injector/ImpactT.in"
os.path.exists(infile)

dat = parse_impact_input(infile)
header = dat["header"]
lattice = dat["lattice"]

In [ ]:
layout = layout_plot(lattice)
show(layout)

# Output Plotting

In [ ]:
from bokeh.plotting import figure
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Range1d
from impact.parsers import load_many_fort
from impact.parsers import UNITS

In [ ]:
opath = "templates/lcls_injector/output/"
data = load_many_fort(opath, verbose=True)
ds = ColumnDataSource(data)

In [ ]:
def plot1(y_axis, x_axis="mean_z", source=ds):
    yunit = UNITS[y_axis]
    ylabel = y_axis + " (" + yunit + ")"

    xunit = UNITS[x_axis]
    xlabel = x_axis + " (" + xunit + ")"
    p = figure(width=600, height=200, x_axis_label=xlabel, y_axis_label=ylabel)
    p.line(x_axis, y_axis, source=source, color="red")
    return p


show(plot1("norm_emit_x"))

# Stacking plots

This also links the panning, zooming

In [ ]:
p1 = plot1("sigma_x")
p2 = plot1("norm_emit_x")
zmax = 2
layout.x_range = Range1d(0, zmax)
p1.x_range = Range1d(0, zmax)

layout.width = p1.width

# Link panning, zooming
layout.x_range = p1.x_range
p2.x_range = p1.x_range

show(column(p1, p2, layout))